In [1]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 2.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import tldextract
from gensim.models import Word2Vec
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re
from urllib.parse import urlparse, parse_qs
import ipaddress

In [3]:
class URLFeatureExtractor:
    def __init__(self):
        self.suspicious_keywords = [
            'login', 'verify', 'update', 'account', 'secure', 'banking',
            'signin', 'confirm', 'password', 'credential', 'security'
        ]
        self.suspicious_tlds = ['.tk', '.ml', '.ga', '.cf', '.gq', '.xyz']

    def extract_features(self, url):
        """Extract features from a single URL"""
        features = {}
        parsed_url = urlparse(url.lower())

        # Basic URL Characteristics
        features['url_length'] = len(url)
        features['domain_length'] = len(parsed_url.netloc)
        features['path_length'] = len(parsed_url.path)
        features['is_https'] = 1 if parsed_url.scheme == 'https' else 0

        # Special characters count
        features['special_char_count'] = sum(url.count(c) for c in "@#$%^&*()+=[]{}|;:,.<>?")

        # Check if domain is an IP address
        features['is_ip_address'] = self._is_ip_address(parsed_url.netloc)

        # Suspicious Keywords Count
        features['suspicious_keyword_count'] = self._count_suspicious_keywords(url)

        # Top-level domain check for suspicious TLDs
        features['has_suspicious_tld'] = self._check_suspicious_tld(parsed_url.netloc)

        # URL structure features
        features['directory_depth'] = parsed_url.path.count('/')
        features['query_param_count'] = len(parse_qs(parsed_url.query))
        features['fragment_length'] = len(parsed_url.fragment)

        # Additional features
        features['has_subdomain'] = 1 if parsed_url.netloc.count('.') > 1 else 0
        features['domain_has_numbers'] = 1 if any(char.isdigit() for char in parsed_url.netloc) else 0
        features['query_param_length'] = sum(len(val) for key, val in parse_qs(parsed_url.query).items())
        features['subdomain_count'] = parsed_url.netloc.count('.') - 1
        features['uses_ip_in_url'] = 1 if parsed_url.netloc.replace('.', '').isdigit() else 0
        features['long_path'] = 1 if len(parsed_url.path) > 50 else 0
        features['has_hyphens_in_domain'] = 1 if '-' in parsed_url.netloc else 0
        features['has_at_symbol'] = 1 if '@' in url else 0

        # Synthetic phishing-specific patterns
        features['has_misspelled_domain'] = self._has_misspelled_domain(parsed_url.netloc)
        features['is_shortened_url'] = self._is_shortened_url(url)

        # Domain extraction without status code
        features['domain_name'], features['tld'] = self._extract_domain_tld(url)

        return features

    def _is_ip_address(self, domain):
        """Check if the domain is an IP address"""
        try:
            ipaddress.ip_address(domain.split(':')[0])
            return 1
        except ValueError:
            return 0

    def _count_suspicious_keywords(self, url):
        """Count suspicious keywords in the URL"""
        return sum(1 for keyword in self.suspicious_keywords if keyword in url)

    def _check_suspicious_tld(self, domain):
        """Check for suspicious top-level domains"""
        return int(any(domain.endswith(tld) for tld in self.suspicious_tlds))

    def _has_misspelled_domain(self, domain):
        misspelled_patterns = [
            r"0{1,}o",
            r"1{1,}l",
            r"3{1,}e",
            r"1{1,}i",
            r"(.)\1{2,}",
            r"faecbook",
            r"gogle",
            r"(o{2,}|g{2,}|e{2,})",
            r"ht{1,}:\/\/",
            r"g{2,}le",
            r"fa{2,}cebook",
            r"0o{1,}gle",
            r"t{1,}witter",
        ]

        return int(any(re.search(pattern, domain) for pattern in misspelled_patterns))

    def _is_shortened_url(self, url):
        """Check if the URL is a shortened link"""
        shortened_domains = [
            'bit.ly', 'tinyurl.com', 'goo.gl', 't.co', 'ow.ly', 'is.gd', 'buff.ly', 'adf.ly'
        ]
        domain = urlparse(url).netloc
        return int(domain in shortened_domains)

    def _extract_domain_tld(self, url):
        """Extract domain name and TLD"""
        extracted = tldextract.extract(url)
        domain_name = extracted.domain
        tld = extracted.suffix
        return domain_name, tld

In [4]:
class PhishingURLDetector:
    def __init__(self):
        self.word2vec_model = None
        self.classifier = None
        self.label_encoder = None
        self.feature_extractor = URLFeatureExtractor()
        self.label_encoder = LabelEncoder()
        self.label_map = {0: 'phishing', 1: 'safe'}  # Fixed label mapping to match your data
        self.vector_size = 100
        self.tld_encoder = LabelEncoder()  # Separate encoder for TLDs

    def save_models(self, model_path='phishing_detector.joblib'):
        """Save all trained models and encoders"""
        if not all([self.word2vec_model, self.classifier, self.label_encoder, self.tld_encoder]):
            raise ValueError("Models not trained! Call train() first.")

        models = {
            'word2vec': self.word2vec_model,
            'classifier': self.classifier,
            'label_encoder': self.label_encoder,
            'tld_encoder': self.tld_encoder
        }
        joblib.dump(models, model_path)

    def load_models(self, model_path='phishing_detector.joblib'):
        """Load all trained models and encoders"""
        try:
            models = joblib.load(model_path)
            self.word2vec_model = models['word2vec']
            self.classifier = models['classifier']
            self.label_encoder = models['label_encoder']
            self.tld_encoder = models['tld_encoder']
        except Exception as e:
            raise ValueError(f"Error loading models: {str(e)}")

    def train(self, urls, labels):
        """Train the complete model pipeline"""
        if not isinstance(urls, (pd.Series, list)) or not isinstance(labels, (pd.Series, list, np.ndarray)):
            raise ValueError("URLs and labels must be pandas Series, lists, or numpy arrays")

        # Convert inputs to numpy arrays
        urls = np.array(urls)
        labels = np.array(labels)

        # Fit label encoder
        self.label_encoder.fit(labels)
        labels_encoded = self.label_encoder.transform(labels)

        # Train Word2Vec model on domains
        tokenized_domains = [list(str(tldextract.extract(url).domain).lower())
                           for url in urls]
        self.word2vec_model = Word2Vec(
            sentences=tokenized_domains,
            vector_size=self.vector_size,
            window=5,
            min_count=1,
            workers=4
        )

        # Extract and encode TLDs
        tlds = [tldextract.extract(url).suffix for url in urls]
        self.tld_encoder.fit(tlds)

        # Prepare features
        X = self._prepare_features(urls)

        # Train stacking classifier
        base_models = [
            ('rf', RandomForestClassifier(
                n_estimators=183,
                max_depth=21,
                min_samples_split=7,
                min_samples_leaf=4,
                max_features='sqrt',
                random_state=42
            )),
            ('gb', GradientBoostingClassifier(
                n_estimators=50,
                random_state=42
            ))
        ]

        meta_model = LogisticRegression(random_state=42)
        self.classifier = StackingClassifier(
            estimators=base_models,
            final_estimator=meta_model,
            cv=3
        )

        try:
            self.classifier.fit(X, labels_encoded)
        except Exception as e:
            raise ValueError(f"Error training classifier: {str(e)}")

    def predict(self, url):
        """Predict whether a single URL is phishing or safe"""
        if not all([self.word2vec_model, self.classifier, self.label_encoder, self.tld_encoder]):
            raise ValueError("Models not trained! Call train() or load_models() first.")

        try:
            # Prepare features for the single URL
            features = self._prepare_single_url_features(url)

            # Make prediction
            prediction = self.classifier.predict(features)[0]
            probability = self.classifier.predict_proba(features)[0]

            # Map prediction to label
            label = self.label_map.get(prediction, 'Unknown')

            # Note: probability[0] is phishing probability and probability[1] is safe probability
            # because 0 means phishing and 1 means safe in your data
            return {
                'url': url,
                'prediction': label,
                'is_phishing': label == 'phishing',
                'confidence': max(probability),
                'probability_phishing': probability[0],  # Fixed probability mapping
                'probability_safe': probability[1]       # Fixed probability mapping
            }
        except Exception as e:
            raise ValueError(f"Error making prediction: {str(e)}")

    def _prepare_single_url_features(self, url):
        """Prepare features for a single URL"""
        try:
            # Extract base features
            features = self.feature_extractor.extract_features(url)

            # Get domain embedding
            domain = str(features['domain_name']).lower()
            domain_tokens = list(domain)

            # Calculate domain embedding
            valid_tokens = [token for token in domain_tokens if token in self.word2vec_model.wv]
            if valid_tokens:
                domain_embedding = np.mean([self.word2vec_model.wv[token] for token in valid_tokens], axis=0)
            else:
                domain_embedding = np.zeros(self.vector_size)

            # Prepare feature dictionary
            feature_dict = {
                'url_length': features['url_length'],
                'domain_length': features['domain_length'],
                'path_length': features['path_length'],
                'is_https': features['is_https'],
                'special_char_count': features['special_char_count'],
                'is_ip_address': features['is_ip_address'],
                'suspicious_keyword_count': features['suspicious_keyword_count'],
                'has_suspicious_tld': features['has_suspicious_tld'],
                'directory_depth': features['directory_depth'],
                'query_param_count': features['query_param_count'],
                'fragment_length': features['fragment_length'],
                'has_subdomain': features['has_subdomain'],
                'domain_has_numbers': features['domain_has_numbers'],
                'query_param_length': features['query_param_length'],
                'subdomain_count': features['subdomain_count'],
                'uses_ip_in_url': features['uses_ip_in_url'],
                'long_path': features['long_path'],
                'has_hyphens_in_domain': features['has_hyphens_in_domain'],
                'has_at_symbol': features['has_at_symbol'],
                'has_misspelled_domain': features['has_misspelled_domain'],
                'is_shortened_url': features['is_shortened_url']
            }

            # Add TLD encoding
            feature_dict['tld_encoded'] = self._encode_tld(features['tld'])

            # Add embedding features
            for i, val in enumerate(domain_embedding):
                feature_dict[f'embedding_{i}'] = val

            return pd.DataFrame([feature_dict])

        except Exception as e:
            raise ValueError(f"Error preparing features: {str(e)}")

    def _encode_tld(self, tld):
        """Encode TLD, handling unknown values"""
        try:
            return self.tld_encoder.transform([str(tld)])[0]
        except ValueError:
            return -1

    def _prepare_features(self, urls):
        """Prepare features for a list of URLs"""
        features_list = []
        for url in urls:
            try:
                features = self._prepare_single_url_features(url)
                features_list.append(features)
            except Exception as e:
                print(f"Warning: Error processing URL {url}: {str(e)}")
                continue

        if not features_list:
            raise ValueError("No valid features could be extracted from URLs")

        return pd.concat(features_list, ignore_index=True)

In [5]:
df = pd.read_csv('url.csv')  # Loading Dataset
print(f"Dataset shape: {df.shape}")
print("\nSample of the data:")
print(df.head())

Dataset shape: (235810, 2)

Sample of the data:
                                  URL  label
0    https://www.southbankmosaics.com      1
1            https://www.uni-mainz.de      1
2      https://www.voicefmradio.co.uk      1
3         https://www.sfnmjournal.com      1
4  https://www.rewildingargentina.org      1


In [6]:
df_subset = df.iloc[-10000:].reset_index(drop=True)
X_train, X_test, y_train, y_test = train_test_split(
    df_subset['URL'],
    df_subset['label'],
    test_size=0.2,
    random_state=42
)
print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

Training samples: 8000
Testing samples: 2000


In [7]:
# Initialize the detector
detector = PhishingURLDetector()

# Train using your existing dataset
detector.train(df['URL'], df['label'])

# Save the trained models
detector.save_models('phishing_detector.joblib')

In [8]:
# Eighth cell - Test the model
# Load the model (optional if you've just trained it)
detector = PhishingURLDetector()
detector.load_models('phishing_detector.joblib')

In [9]:
# Test URLs
test_urls = [
    "https://www.google.com",
    "https://facebook.com",
    "http://suspicious-login.tk/verify",
    "https://www.go0gle.com/"
]

print("Testing URLs:")
for url in test_urls:
    result = detector.predict(url)
    print(f"\nURL: {result['url']}")
    print(f"Prediction: {result['prediction']}")
    print(f"Confidence: {result['confidence']:.2f}")
    print(f"Probability of being phishing: {result['probability_phishing']:.2f}")
    print(f"Probability of being safe: {result['probability_safe']:.2f}")

Testing URLs:

URL: https://www.google.com
Prediction: safe
Confidence: 0.99
Probability of being phishing: 0.01
Probability of being safe: 0.99

URL: https://facebook.com
Prediction: phishing
Confidence: 0.97
Probability of being phishing: 0.97
Probability of being safe: 0.03

URL: http://suspicious-login.tk/verify
Prediction: phishing
Confidence: 1.00
Probability of being phishing: 1.00
Probability of being safe: 0.00

URL: https://www.go0gle.com/
Prediction: phishing
Confidence: 1.00
Probability of being phishing: 1.00
Probability of being safe: 0.00


In [10]:
from sklearn.metrics import classification_report, confusion_matrix

# Make predictions on test set
y_pred = []
for url in X_test:
    result = detector.predict(url)
    y_pred.append(1 if result['prediction'] == 'safe' else 0)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       853
           1       1.00      1.00      1.00      1147

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000


Confusion Matrix:
[[ 850    3]
 [   0 1147]]
